<a href="https://colab.research.google.com/github/ShokuninSan/deep-q-learning-from-paper-to-code/blob/master/09_frozen_lake_q_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gym
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import torch as T

In [ ]:
%matplotlib inline

# FrozenLake using Naive Deep Q-learning algorithm

## The `DQN`

In [ ]:
class DQN(nn.Module):
    
    def __init__(self, n_input, n_hidden, n_classes,
                 loss=nn.MSELoss(),
                 optimizer=optim.Adam,
                 activation=lambda x: x,
                 lr=0.001):
        super(DQN, self).__init__()
        
        self.fc1 = nn.Linear(n_input, n_hidden)
        self.fc2 = nn.Linear(n_hidden, n_classes)
        
        self.activation = activation
        self.loss = loss
        self.optimizer = optimizer(self.parameters(), lr=lr)       
        self.device = T.device('cuda' if T.cuda.is_available() else 'cpu')
        self.to(self.device)
        
    def forward(self, X):
        X_ = T.FloatTensor(X).to(self.device)
        fc1 = F.relu(self.fc1(X_))
        fc2 = self.fc2(fc1)
        return fc2
    
    def learn(self, input_values, target_values):
        self.optimizer.zero_grad()
        
        input_values = T.FloatTensor(input_values).to(self.device)
        target_values = T.FloatTensor(target_values).to(self.device)
        
        predicted_values = self.activation(self.forward(input_values))
        
        loss = self.loss(predicted_values, target_values)
        loss.backward()
        
        self.optimizer.step()

### Test the DQN and approximate a binary AND operator

In [ ]:
net = DQN(2, 4, 1, loss=nn.BCELoss(), optimizer=optim.Adam, lr=0.1, activation=T.sigmoid)

In [ ]:
X = np.array([
    [1, 1],
    [0, 1],
    [0, 0],
    [1, 0]
]*10)

Y = [[1], [0], [0], [0]]*10

In [ ]:
for x, y in zip(X, Y):
    net.learn(x, y)

In [ ]:
T.sigmoid(net.forward([1,1])).item()

In [ ]:
T.sigmoid(net.forward([0,0])).item()

In [ ]:
T.sigmoid(net.forward([0,1])).item()

In [ ]:
T.sigmoid(net.forward([1,0])).item()

## The `Agent`

Reuse the `Agent` implementation of the Q-learning example and substitute the dict by a `DQN`.

In [ ]:
class Agent:

    def __init__(self, n_observations, n_actions, gamma=0.99):
        self.n_actions = n_actions
        self.n_observations = n_observations
        self.gamma = gamma
        self.Q = DQN(n_observations, 128, n_actions)

    def select_action(self, state, epsilon):
        action = None
        if np.random.rand() < epsilon:
            # do random action selection
            action = np.random.randint(self.n_actions)
        else:
            # do greedy action selection
            action_values = self.Q.forward(self._one_hot_encode(state)).detach().numpy()
            action = np.argmax(action_values)

        return action
    
    # TODO implement learning based on Bellman equation: 
    def learn(self, state, action, reward, new_state):
        """
        Bellman
        
        Q(s,a) = Q(s,a) + alpha * (reward + gamma * (max Q(s',a) - Q(s,a)))
        """
        Q = self.Q.forward(self._one_hot_encode(state))
        
        # TODO use a target network
        Q_ = self.Q.forward(self._one_hot_encode(new_state))
        
        expected_values = Q + reward + self.gamma * Q_ - Q
        
        self.Q.learn(self._one_hot_encode(state), expected_values)
        
    def _one_hot_encode(self, state):
        state_ = np.zeros(shape=(self.n_observations,))
        state_[state] = 1
        return state_

In [ ]:
N_EPISODES = 10_000
ALPHA = 0.001
GAMMA = 0.99
EPSILON_START = 1.0
EPSILON_END = 0.01
EPSILON_END_TARGET = 2500

In [ ]:
rewards = []
avg_100_rewards = []
env = gym.make('FrozenLake-v0')
agent = Agent(env.observation_space.n, env.action_space.n)
epsilon = EPSILON_START

In [ ]:
for episode in range(N_EPISODES):

    state = env.reset()

    while True:
        
        action = agent.select_action(state, epsilon)

        new_state, reward, is_done, _ = env.step(action)
        
        agent.learn(state, action, reward, new_state)

        if is_done:
            rewards.append(reward) 
            break

    epsilon = np.max([epsilon - (EPSILON_START/EPSILON_END_TARGET), EPSILON_END]) 

    if episode % 100 == 0:
        avg_reward = np.mean(rewards[-100:])
        avg_100_rewards.append(avg_reward)

    if episode % 1000 == 0:
        print(f'Average reward over last episodes was {avg_100_rewards[-1]}, '
              f'epsilon: {epsilon}')


In [ ]:
plt.plot(avg_100_rewards)

In [ ]:
agent.Q